# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CombinedHeatPower_OrganicRankineCycle executed by RUN — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CombinedHeatPower_OrganicRankineCycle executed standalone — using directory: {current_dir}")


Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 CombinedHeatPower_OrganicRankineCycle executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC


In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'CHP-Organic Rankine Cycle')

# 2. Input parameters

In [4]:
# DASHBOARD - VALUE INPUT - SPECIFIC DATA
Power = energysystem['Thermal capacity [kW]'] 
EfficiencyW = energysystem['Electrical efficiency based on LHV (between 0 and 1)']
EfficiencyQ = energysystem['Heat efficiency based on LHV (between 0 and 1)']
ElectricEnergyproduced = energysystem['Electric energy produced in a year (exported and consumed) [kWh /year]']
ThermalEnergyproduced_primary = energysystem['Thermal energy produced used [kWh /year]']
Qused = 1
WasteO = energysystem["Biomass used as a waste ['yes' or 'no']"]
SupplyTemperature = energysystem['Supply temperature [°C]']
ReturnTemperature = energysystem['Return temperature [°C]']
EquivalentHours = ThermalEnergyproduced_primary / Power

In [5]:
HHV = 18.9  # In MJ per kg
LHV = 18 #MJ per kg

Lifespan = np.where(
    Power * (EfficiencyW / EfficiencyQ)  < 25,
    40000 / EquivalentHours,
    100000 / EquivalentHours
)

# ADDITIONAL CALCULATIONS
EfficiencyTot = EfficiencyW + EfficiencyQ
EnergyProducedLHV = (Power * EquivalentHours * RSP) / EfficiencyQ
EnergyProducedHHV = EnergyProducedLHV * (HHV / LHV)
TotalW = EnergyProducedLHV * EfficiencyW
TotalQ = EnergyProducedLHV * EfficiencyQ * Qused

In [6]:
# MULTIFUNCTIONALITY AND ALLOCATION OF output
if Multifunctionality == 'Default': 
    ExternalTemperature = ExternalTemperature + 273.15 #K
    SupplyTemperature = SupplyTemperature + 273.15 #K
    ReturnTemperature = ReturnTemperature + 273.15 #K
    CarnotTemperature = (SupplyTemperature - ReturnTemperature) / np.log(SupplyTemperature / ReturnTemperature)
    ThermalEnergyproduced = EnergyProducedHHV
    ElectricEnergyproduced = (EnergyProducedHHV / EfficiencyQ) * EfficiencyW 
    # AFQ calculation
    AFQ_ORC = (ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature))) / (ElectricEnergyproduced + ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature)))
    AFQ_ORC = np.array(AFQ_ORC)
    
    # AFW calculation
    AFW_ORC = (1 - AFQ_ORC)
    
else:
    fwecr = 0.7
    fweexp = fwecr
    
    # Calculations
    InletEnergyCogenmode = EnergyProducedHHV * EfficiencyTot
    AFW_ORC = (InletEnergyCogenmode * fwecr - (ElectricEnergyproduced * fweexp)) / EnergyProducedHHV
    AFQ_ORC = 1 - AFW_ORC 

if SupplyTemperature.eq(0).all() and ReturnTemperature.eq(0).all():
    AFQ_ORC = np.zeros_like(AFQ_ORC)
    AFW_ORC = np.zeros_like(AFW_ORC)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [7]:
if Power.all(0) < 500:
    Power_op = np.array(Power)
    ORC_dustCollector = np.full(len(Power_op), 1) * AFQ_ORC
    ORC_furnace = Power_op/1000 * AFQ_ORC
    ORC_CHP = Power_op/200 * AFQ_ORC

elif Power.all(0) >= 500 and Power.all(0) < 2000:
    Power_op = np.array(Power)    
    ORC_dustCollector = np.full(len(Power_op), 1) * AFQ_ORC
    ORC_furnace = (Power_op/1000) * AFQ_ORC
    ORC_CHP = (Power_op/1000) * AFQ_ORC

else:
    Power_op = np.array(Power)    
    i = np.ceil(Power_op/2000)
    ORC_dustCollector = (np.full(len(Power_op), 1) * i * AFQ_ORC).flatten()
    ORC_furnace = (Power_op/1000) * i * AFQ_ORC
    ORC_CHP = (Power_op/3000) * i * AFQ_ORC

## 3.2 Module A4

In [8]:
if Power.all(0) < 500:
    mass = 385977 #kg  ecoinvent data
else:
    mass = 403394.5 #kg ecoinvent data

mass = np.full(len(Power), mass)
ORC_Truk_16_32 = ((500 + 200)*(mass)/1000) * AFQ_ORC
ORC_Light_Truk = (50*(mass)/1000) * AFQ_ORC

## 3.3 Module B2

In [9]:
mass = np.full(len(Power), mass)
ORC_Light_Truk_B2 = (50 * (Lifespan/2) * 0.01 * (mass/1000)) * AFQ_ORC

## 3.4 Module B4

In [10]:
ORCNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFORC = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [11]:
x = 1
TotEnergyNeeds = np.array([EnergyProducedHHV])
TotEnergyNeeds = TotEnergyNeeds.flatten()
y = (TotEnergyNeeds * 3.6).flatten() * AFQ_ORC

ORC_Ammonia1 = (0.0000000082 * x * y).flatten()
ORC_Chemical = (5.84999999999999E-06 * x * y).flatten()
ORC_Chlorine = (0.000000328 * x * y).flatten()
ORC_Oil = (0.00000328 * x * y).flatten()
ORC_NOx = (0.0000269859002222463 * x * y).flatten()
ORC_Sodium = (0.0000041 * x * y).flatten()
ORC_Water = (0.000786999999999999 * x * y).flatten()
if WasteO.all() == 'yes':
   ORC_Wood = 0
else:
    ORC_Wood = (0.000529100529100529 * x * y).flatten()

ORC_Acetaldehyde = (4.57261087099173E-08 * x * y).flatten()
ORC_Ammonia2 = (0.0000127433417716163 * x * y).flatten()
ORC_Benzene = (6.82143588951226E-07 * x * y).flatten()
ORC_Benzeneethyl = (2.24882501852052E-08 * x * y).flatten()
ORC_Benzenehexachloro = (5.39718004444926E-15 * x * y).flatten()
ORC_Benzoapyrene = (3.74804169753421E-10 * x * y).flatten()
ORC_Bromine = (4.49765003704105E-08 * x * y).flatten()
ORC_Cadmium = (5.24725837654789E-10 * x * y).flatten()
ORC_Calcium = (4.38520878611502E-06 * x * y).flatten()
ORC_Carbondioxidebiogenic = ((0.0719624005926568) * x * y).flatten()
ORC_Carbonmonoxidebiogenic = ((0.0000749608339506842) * x * y).flatten()
ORC_Chlorine = (1.34929501111231E-07 * x * y).flatten()
ORC_Chromium = (2.96844902444709E-09 * x * y).flatten()
ORC_ChromiumVI = (2.99843335802737E-11 * x * y).flatten()
ORC_Copper = (1.64913834691505E-08 * x * y).flatten()
ORC_Dinitrogenmonoxide = (1.72409918086574E-06 * x * y).flatten()
ORC_Dioxin = (2.32378585247121E-14 * x * y).flatten()
ORC_Fluorine = (3.74804169753421E-08 * x * y).flatten()
ORC_Formaldehyde = (9.74490841358894E-08 * x * y).flatten()
ORC_Hydrocarbonsaliphaticalkanesunspecified = (6.82143588951226E-07 * x * y).flatten()
ORC_Hydrocarbonsaliphaticunsaturated = (2.32378585247121E-06 * x * y).flatten()
ORC_Lead = (1.8740208487671E-08 * x * y).flatten()
ORC_Magnesium = (2.69859002222463E-07 * x * y).flatten()
ORC_Manganese = (1.27433417716163E-07 * x * y).flatten()
ORC_Mercury = (2.24882501852052E-10 * x * y).flatten()
ORC_Methanebiogenic = (8.9953000740821E-07 * x * y).flatten()
ORC_mXylene = (8.9953000740821E-08 * x * y).flatten()
ORC_Nickel = (4.49765003704105E-09 * x * y).flatten()
ORC_Nitrogenoxides = (0.0000599686671605473 * x * y).flatten()
ORC_NMVOC = (2.09890335061916E-06 * x * y).flatten()
ORC_PAH = (8.32065256852594E-09 * x * y).flatten()
ORC_Particulates = (3.74804169753421E-06 * x * y).flatten()
ORC_Phenolpentachloro = (6.07182755000542E-12 * x * y).flatten()
ORC_Phosphorus = (2.24882501852052E-07 * x * y).flatten()
ORC_Potassium = (0.0000175408351444601 * x * y).flatten()
ORC_Sodium = (9.74490841358894E-07 * x * y).flatten()
ORC_Sulfurdioxide = (0.0000018740208487671 * x * y).flatten()
ORC_Toluene = (2.24882501852052E-07 * x * y).flatten()
ORC_Watermc = (0.0000436272053592982 * x * y).flatten()
ORC_Zinc = (2.24882501852052E-07 * x * y).flatten()
ORC_Solidwaste = (0.00000328 * x * y).flatten()
ORC_Mineraloilwaste = (0.00000328 * x * y).flatten()
ORC_Wastewater = (0.000000787 * x * y).flatten()

## 3.6 Module C2

In [12]:
if Power.all(0) < 500:
    mass = 385977 #kg ecoinvent data
else:
    mass = 403394.5 #kg ecoinvent data

mass = np.full(len(Power), mass)
ORC_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [13]:
ORC_EOL_aluminiumDustcollector = (12 * Power) / 200
ORC_EOL_ceramictileDustcollector = (42 * Power) / 200
ORC_EOL_copperDustcollector = (37 * Power) / 200
ORC_EOL_drawingpipeSteelDustcollector = (83 * Power) / 200
ORC_EOL_electronicsDustcollector = (5 * Power) / 200
ORC_EOL_glasswoolmatDustcollector = (55 * Power) / 200
ORC_EOL_HDPEDustcollector = (31 * Power) / 200
ORC_EOL_sheetrollingsteelDustcollector = (83 * Power) / 200
ORC_EOL_chromiumsteelDustcollector = (810 * Power) / 200
ORC_EOL_lowalloyedsteelDustcollector = (690 * Power) / 200
ORC_EOL_alkydpaintFurnace = (12 * Power) / 200
ORC_EOL_aluminiumFurnace = (104 * Power) / 200
ORC_EOL_castironFurnace = (1115 * Power) / 200
ORC_EOL_concreteFurnace = (134400 * Power) / 200
ORC_EOL_copperFurnace = (46 * Power) / 200
ORC_EOL_drawingpipeFurnace = (37410 * Power) / 200
ORC_EOL_electronicsFurnace = (12 * Power) / 200
ORC_EOL_ironnickelchromiumalloyFurnace = (528 * Power) / 200
ORC_EOL_lubricatingoilFurnace = (32 * Power) / 200
ORC_EOL_HDPEFurnace = (18 * Power) / 200
ORC_EOL_polystyreneFurnace = (361 * Power) / 200
ORC_EOL_claybrickFurnace = (41220 * Power) / 200
ORC_EOL_sheetrollingsteelFurnace = (37410 * Power) / 200
ORC_EOL_chromiumsteelFurnace = (70 * Power) / 200
ORC_EOL_lowalloyedsteelFurnace = (74820 * Power) / 200
ORC_EOL_stonewoolFurnace = (1846 * Power) / 200

if Power.all() < 500:
    ORC_EOL_copperCHP = (67 * Power) / 200
    ORC_EOL_electronicsCHP = (30 * Power) / 200
    ORC_EOL_perfluoropentaneCHP = (1200 * Power) / 200
    ORC_EOL_HDPECHP = (18 * Power) / 200
    ORC_EOL_chromiumsteelCHP = (7950 * Power) / 200
    ORC_EOL_lowalloyedsteelCHP = (45050 * Power) / 200
    ORC_EOL_stonewoolCHP = (410 * Power) / 200
else:
    ORC_EOL_copperCHP = (87.1 * Power) / 200
    ORC_EOL_electronicsCHP = (39 * Power) / 200
    ORC_EOL_perfluoropentaneCHP = (1560 * Power) / 200
    ORC_EOL_HDPECHP = (23.4 * Power) / 200
    ORC_EOL_chromiumsteelCHP = (10335 * Power) / 200
    ORC_EOL_lowalloyedsteelCHP = (58565 * Power) / 200
    ORC_EOL_stonewoolCHP = (533 * Power) / 200

## 3.8 Module C4

In [14]:
ORC_EOL_landfill_aluminiumDustcollector = ORC_EOL_aluminiumDustcollector * LandfillMetals
ORC_EOL_landfill_ceramictileDustcollector = ORC_EOL_ceramictileDustcollector * LandfillClaybrick
ORC_EOL_landfill_copperDustcollector = ORC_EOL_copperDustcollector * LandfillMetals
ORC_EOL_landfill_drawingpipesteelDustcollector = ORC_EOL_drawingpipeSteelDustcollector * LandfillMetals
ORC_EOL_inc_electronicsDustcollector = ORC_EOL_electronicsDustcollector * IncinerationElectronics #kg
ORC_EOL_landfill_electronicsDustcollector = ORC_EOL_electronicsDustcollector * LandfillElectronics #kg
ORC_EOL_landfill_glasswoolmatDustcollector = ORC_EOL_glasswoolmatDustcollector * LandfillGlass #kg
ORC_EOL_inc_HDPEDustcollector = ORC_EOL_HDPEDustcollector * IncinerationPEHD
ORC_EOL_landfill_HDPEDustcollector = ORC_EOL_HDPEDustcollector * LandfillPEHD
ORC_EOL_landfill_sheetrollingsteelDustcollector = ORC_EOL_sheetrollingsteelDustcollector * LandfillMetals
ORC_EOL_landfill_chromiumsteelDustcollector = ORC_EOL_chromiumsteelDustcollector * LandfillMetals2
ORC_EOL_landfill_lowalloyedsteelDustcollector = ORC_EOL_lowalloyedsteelDustcollector * LandfillMetals
ORC_EOL_inc_alkydpaintFurnace = ORC_EOL_alkydpaintFurnace
ORC_EOL_landfill_aluminiumFurnace = ORC_EOL_aluminiumFurnace * LandfillMetals
ORC_EOL_landfill_castironFurnace = ORC_EOL_castironFurnace * LandfillMetals2
ORC_EOL_landfill_concreteFurnace = ORC_EOL_concreteFurnace * LandfillConcrete
ORC_EOL_landfill_copperFurnace = ORC_EOL_copperFurnace * LandfillMetals
ORC_EOL_landfill_drawingpipesteelFurnace = ORC_EOL_drawingpipeFurnace * LandfillMetals
ORC_EOL_inc_electronicsFurnace = ORC_EOL_electronicsFurnace * IncinerationElectronics #kg
ORC_EOL_landfill_electronicsFurnace = ORC_EOL_electronicsFurnace * LandfillElectronics #kg
ORC_EOL_landfill_ironnickelchromiumalloyFurnace = ORC_EOL_ironnickelchromiumalloyFurnace * LandfillMetals
ORC_EOL_inc_lubricatingoilFurnace = ORC_EOL_lubricatingoilFurnace * IncinerationLubricatingOil #kg
ORC_EOL_inc_HDPEFurnace = ORC_EOL_HDPEFurnace * IncinerationPEHD
ORC_EOL_landfill_HDPEFurnace = ORC_EOL_HDPEFurnace * LandfillPEHD
ORC_EOL_inc_polystyreneFurnace = ORC_EOL_polystyreneFurnace * IncinerationEPS
ORC_EOL_landfill_claybrickFurnace = ORC_EOL_claybrickFurnace * LandfillClaybrick
ORC_EOL_landfill_sheetrollingsteelFurnace = ORC_EOL_sheetrollingsteelFurnace * LandfillMetals
ORC_EOL_landfill_chromiumsteelFurnace = ORC_EOL_chromiumsteelFurnace * LandfillMetals2
ORC_EOL_landfill_lowalloyedsteelFurnace = ORC_EOL_lowalloyedsteelFurnace * LandfillMetals
ORC_EOL_landfill_stonewoolFurnace = ORC_EOL_stonewoolFurnace * LandfillMineralWool
ORC_EOL_landfill_copperCHP = ORC_EOL_copperCHP * LandfillMetals
ORC_EOL_inc_electronicsCHP = ORC_EOL_electronicsCHP * IncinerationElectronics #kg
ORC_EOL_landfill_electronicsCHP = ORC_EOL_electronicsCHP * LandfillElectronics #kg
ORC_EOL_inc_perfluoropentaneCHP = ORC_EOL_perfluoropentaneCHP * IncinerationHazardous #kg
ORC_EOL_inc_HDPECHP = ORC_EOL_HDPECHP * IncinerationPEHD
ORC_EOL_landfill_HDPEDCHP = ORC_EOL_HDPECHP * LandfillPEHD
ORC_EOL_landfill_chromiumsteelCHP = ORC_EOL_chromiumsteelCHP * LandfillMetals2
ORC_EOL_landfill_lowalloyedsteelCHP = ORC_EOL_lowalloyedsteelCHP * LandfillMetals
ORC_EOL_landfill_stonewoolCHP = ORC_EOL_stonewoolCHP * LandfillMineralWool

## 3.9 Module D1 (related to the export of secondary materials)

In [15]:
ORC_EOL_recycling_aluminiumDustcollector = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_aluminiumDustcollector) - (RecycledContentMetals * ORC_EOL_aluminiumDustcollector)
ORC_EOL_primary_aluminiumDustcollector = ((RecycledContentMetals * ORC_EOL_aluminiumDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_aluminiumDustcollector))* SRmetals

ORC_EOL_recycling_copperDustcollector = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperDustcollector) - (RecycledContentMetals * ORC_EOL_copperDustcollector)
ORC_EOL_primary_copperDustcollector = ((RecycledContentMetals * ORC_EOL_copperDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperDustcollector))* SRmetals

ORC_EOL_recycling_drawingpipeDustcollector = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_drawingpipeSteelDustcollector) - (RecycledContentMetals * ORC_EOL_drawingpipeSteelDustcollector)
ORC_EOL_primary_drawingpipeDustcollector = ((RecycledContentMetals * ORC_EOL_drawingpipeSteelDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_drawingpipeSteelDustcollector))* SRmetals

ORC_EOL_recycling_HDPEDustcollector = (RecyclingPEHD * RecyclingEfficiencyPlastics * ORC_EOL_HDPEDustcollector) - (RecycledContentPlastic * ORC_EOL_HDPEDustcollector)
ORC_EOL_primary_HDPEDustcollector = ((RecycledContentPlastic * ORC_EOL_HDPEDustcollector) - (RecycledContentPlastic * ORC_EOL_HDPEDustcollector))* SRinert

ORC_EOL_recycling_sheetrollingsteelDustcollector = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_sheetrollingsteelDustcollector) - (RecycledContentMetals * ORC_EOL_sheetrollingsteelDustcollector)
ORC_EOL_primary_sheetrollingsteelDustcollector = ((RecycledContentMetals * ORC_EOL_sheetrollingsteelDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_sheetrollingsteelDustcollector))* SRmetals

ORC_EOL_recycling_chromiumsteelDustcollector = (RecyclingMetals2 * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelDustcollector) - (RecycledContentMetals * ORC_EOL_chromiumsteelDustcollector)
ORC_EOL_primary_chromiumsteelDustcollector = ((RecycledContentMetals * ORC_EOL_chromiumsteelDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelDustcollector))* SRmetals

ORC_EOL_recycling_lowalloyedsteelDustcollector = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelDustcollector) - (RecycledContentMetals * ORC_EOL_lowalloyedsteelDustcollector)
ORC_EOL_primary_lowalloyedsteelDustcollector = ((RecycledContentMetals * ORC_EOL_lowalloyedsteelDustcollector) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelDustcollector))* SRmetals

ORC_EOL_recycling_aluminiumFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_aluminiumFurnace) - (RecycledContentMetals * ORC_EOL_aluminiumFurnace)
ORC_EOL_primary_aluminiumFurnace = ((RecycledContentMetals * ORC_EOL_aluminiumFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_aluminiumFurnace))* SRmetals

ORC_EOL_recycling_castironFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_castironFurnace) - (RecycledContentMetals * ORC_EOL_castironFurnace)
ORC_EOL_primary_castironFurnace = ((RecycledContentMetals * ORC_EOL_castironFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_castironFurnace))* SRmetals

ORC_EOL_recycling_concreteFurnace = (RecyclingConcrete * RecyclingEfficiencyConcrete * ORC_EOL_concreteFurnace) - (RecycledContentConcrete * ORC_EOL_concreteFurnace)
ORC_EOL_primary_concreteFurnace = ((RecycledContentConcrete * ORC_EOL_concreteFurnace) - (RecyclingConcrete * RecyclingEfficiencyConcrete * ORC_EOL_concreteFurnace))* SRinert

ORC_EOL_recycling_copperFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperFurnace) - (RecycledContentMetals * ORC_EOL_copperFurnace)
ORC_EOL_primary_copperFurnace = ((RecycledContentMetals * ORC_EOL_copperFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperFurnace))* SRmetals

ORC_EOL_recycling_drawingpipesteelFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_drawingpipeFurnace) - (RecycledContentMetals * ORC_EOL_drawingpipeFurnace)
ORC_EOL_primary_drawingpipesteelFurnace = ((RecycledContentMetals * ORC_EOL_drawingpipeFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_drawingpipeFurnace))* SRmetals

ORC_EOL_recycling_ironickelchromiumsteelFurnace = (RecyclingMetals2 * RecyclingEfficiencyMetals * ORC_EOL_ironnickelchromiumalloyFurnace) - (RecycledContentMetals * ORC_EOL_ironnickelchromiumalloyFurnace)
ORC_EOL_primary_ironickelchromiumsteelFurnace = ((RecycledContentMetals * ORC_EOL_ironnickelchromiumalloyFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_ironnickelchromiumalloyFurnace))* SRmetals

ORC_EOL_recycling_HDPEFurnace = (RecyclingPEHD * RecyclingEfficiencyPlastics * ORC_EOL_HDPEFurnace) - (RecycledContentPlastic * ORC_EOL_HDPEFurnace)
ORC_EOL_primary_HDPEFurnace = ((RecycledContentPlastic * ORC_EOL_HDPEFurnace) - (RecycledContentPlastic * ORC_EOL_HDPEFurnace))* SRinert

ORC_EOL_recycling_claybrickFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_claybrickFurnace) - (RecycledContentClaybrick * ORC_EOL_claybrickFurnace)
ORC_EOL_primary_claybrickFurnace = ((RecycledContentClaybrick * ORC_EOL_claybrickFurnace))* SRclaybrick

ORC_EOL_recycling_sheetrollingsteelFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_sheetrollingsteelFurnace) - (RecycledContentMetals * ORC_EOL_sheetrollingsteelFurnace)
ORC_EOL_primary_sheetrollingsteelFurnace = ((RecycledContentMetals * ORC_EOL_sheetrollingsteelFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_sheetrollingsteelFurnace))* SRmetals

ORC_EOL_recycling_chromiumsteelFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelFurnace) - (RecycledContentMetals * ORC_EOL_chromiumsteelFurnace)
ORC_EOL_primary_chromiumsteelFurnace = ((RecycledContentMetals * ORC_EOL_chromiumsteelFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelFurnace))* SRmetals

ORC_EOL_recycling_lowalloyedsteelFurnace = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelFurnace) - (RecycledContentMetals * ORC_EOL_lowalloyedsteelFurnace)
ORC_EOL_primary_lowalloyedsteelFurnace = ((RecycledContentMetals * ORC_EOL_lowalloyedsteelFurnace) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelFurnace))* SRmetals

ORC_EOL_recycling_copperCHP = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperCHP) - (RecycledContentMetals * ORC_EOL_copperCHP)
ORC_EOL_primary_copperCHP = ((RecycledContentMetals * ORC_EOL_copperCHP) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_copperCHP))* SRmetals

ORC_EOL_recycling_HDPECHP = (RecyclingPEHD * RecyclingEfficiencyPlastics * ORC_EOL_HDPECHP) - (RecycledContentPlastic * ORC_EOL_HDPECHP)
ORC_EOL_primary_HDPECHP = ((RecycledContentPlastic * ORC_EOL_HDPECHP) - (RecycledContentPlastic * ORC_EOL_HDPECHP))* SRinert

ORC_EOL_recycling_chromiumsteelCHP = (RecyclingMetals2 * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelCHP) - (RecycledContentMetals * ORC_EOL_chromiumsteelCHP)
ORC_EOL_primary_chromiumsteelCHP = ((RecycledContentMetals * ORC_EOL_chromiumsteelCHP) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_chromiumsteelCHP))* SRmetals

ORC_EOL_recycling_lowalloyedsteelCHP = (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelCHP) - (RecycledContentMetals * ORC_EOL_lowalloyedsteelCHP)
ORC_EOL_primary_lowalloyedsteelCHP = ((RecycledContentMetals * ORC_EOL_lowalloyedsteelCHP) - (RecyclingMetals * RecyclingEfficiencyMetals * ORC_EOL_lowalloyedsteelCHP))* SRmetals


## 3.10 Module D3 (related to the export of energy as result of waste incineration)

In [16]:
ORC_EOL_energyrecoveryheat_electronicsDustcollector = - ORC_EOL_inc_electronicsDustcollector * LHVElectronics * Xincheat
ORC_EOL_energyrecoveryele_electronicsDustcollector = - ORC_EOL_inc_electronicsDustcollector * LHVElectronics * Xincele

ORC_EOL_energyrecoveryheat_HDPEDustcollector = - ORC_EOL_inc_HDPEDustcollector * LHVHDPE * Xincheat
ORC_EOL_energyrecoveryele_HDPEDustcollector = - ORC_EOL_inc_HDPEDustcollector * LHVHDPE * Xincele

ORC_EOL_energyrecoveryheat_alkydpaintFurnace = - ORC_EOL_inc_alkydpaintFurnace * LHVHazardous * Xincheat
ORC_EOL_energyrecoveryele_alkydpaintFurnace = - ORC_EOL_inc_alkydpaintFurnace * LHVHazardous * Xincele

ORC_EOL_energyrecoveryheat_electronicsFurnace = - ORC_EOL_inc_electronicsFurnace * LHVElectronics * Xincheat
ORC_EOL_energyrecoveryele_electronicsFurnace = - ORC_EOL_inc_electronicsFurnace * LHVElectronics * Xincele

ORC_EOL_energyrecoveryheat_lubricatingoilFurnace = - ORC_EOL_inc_lubricatingoilFurnace * LHVHazardous  * Xincheat
ORC_EOL_energyrecoveryele_lubricatingoilFurnace = - ORC_EOL_inc_lubricatingoilFurnace * LHVHazardous  * Xincele

ORC_EOL_energyrecoveryheat_HDPEFurnace = - ORC_EOL_inc_HDPEFurnace * LHVHDPE * Xincheat
ORC_EOL_energyrecoveryele_HDPEFurnace = - ORC_EOL_inc_HDPEFurnace * LHVHDPE * Xincele

ORC_EOL_energyrecoveryheat_polystyreneFurnace = - ORC_EOL_inc_polystyreneFurnace * LHVEPS * Xincheat
ORC_EOL_energyrecoveryele_polystyreneFurnace = - ORC_EOL_inc_polystyreneFurnace * LHVEPS * Xincele

ORC_EOL_energyrecoveryheat_electronics = - ORC_EOL_inc_electronicsCHP * LHVElectronics * Xincheat
ORC_EOL_energyrecoveryele_electronics = - ORC_EOL_inc_electronicsCHP * LHVElectronics * Xincele

ORC_EOL_energyrecoveryheat_perfluoropentaneCHP = - ORC_EOL_inc_perfluoropentaneCHP * LHVHazardous  * Xincheat
ORC_EOL_energyrecoveryele_perfluoropentaneCHP = - ORC_EOL_inc_perfluoropentaneCHP * LHVHazardous  * Xincele

ORC_EOL_energyrecoveryheat_HDPECHP = - ORC_EOL_inc_HDPECHP * LHVHDPE * Xincheat
ORC_EOL_energyrecoveryele_HDPECHP = - ORC_EOL_inc_HDPECHP * LHVHDPE * Xincele

# 4. Data for export

In [17]:
ThermalEnergyProduced_ORC = TotEnergyNeeds